In [2]:
# 1) Install deps
!pip -q install hydra-core hydra-optuna-sweeper optuna torch torchvision

# 2) Upload ZIP (if you haven't already) or fetch from local cell output
#    In this chat, download the provided file and then upload it to Colab.
#    Or if you placed it in /. already, just unzip:
import zipfile, os

zip_path = "./hydra_optuna_cifar.zip"  # <- upload the ZIP here via Colab "Files" sidebar
assert os.path.exists(zip_path), "Upload hydra_optuna_cifar.zip to /. first (Files sidebar)"
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(".")

# 3) Enter the project
%cd ./hydra_optuna_cifar

# 4) Single run (no sweeper): simple override from CLI
!python train.py training.epochs=3 optimizer.lr=3e-4 data.batch_size=64

# 5) Factorial ablation (Hydra multirun enumerates the “outer” design questions)
!python train.py -m model.norm_type=none,batch,layer model.use_residual=true,false data.batch_size=32,64 hpo.seeds=[0,1]

# 6) HPO (Hydra Optuna Sweeper) = "best-of-each" tuning per design
!python train.py -m +experiment=hpo hpo.seeds=[0,1,2]

# (Optional) Programmatic Optuna (per-epoch pruning support):
# Edit conf/config.yaml -> set hpo.mode="optuna_programmatic", then:
# !python train.py

/sfs/gpfs/tardis/home/epx8hh/Documents/MSDS/DS6050/DS6050-DeepLearning/3. Convolutional Neural Networks/hydra_optuna_cifar
==== Resolved Config ====
seed: 0
hpo:
  seeds:
  - 0
  - 1
  mode: none
data:
  root: ./data
  batch_size: 64
  num_workers: 2
  augment: false
  use_fake: false
model:
  input_size: 3072
  n_layers: 3
  hidden_size: 512
  norm_type: batch
  use_residual: true
  init: kaiming_normal
  dropout: 0.3
  num_classes: 10
optimizer:
  name: Adam
  lr: 0.0003
  weight_decay: 0.0001
  momentum: 0.9
training:
  epochs: 3
  device: auto
  early_stop: false

100%|█████████████████████████████████████████| 170M/170M [00:01<00:00, 105MB/s]
/home/epx8hh/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[RESULT] val_acc_mean=0.4657  val_acc_std=0.0083  params=2634250
[2025-09-24 18:38:46,207][HYDRA] Launching 12 jo